In [1]:
import os
import cv2
import zipfile
import sys
import tensorflow as tf
import numpy as np
from PIL import Image

In [2]:
# def unzip_dataset(filepath):
#     if not os.path.exists(filepath):
#         print("%s not Found." % filepath)
    
#     dir = os.path.dirname(filepath)
#     zf = zipfile.ZipFile(filepath, 'r')
#     zf.exactall(dir, )

# get filelist and labels
def get_filelist(ds_dir):
    filelist = []
    labels = []
    
    if not os.path.exists(ds_dir):
        print("%s not Found." % ds_dir)
        return -1
        
    for root, subdirs, files in os.walk(ds_dir):
        for filename in files:
            filepath = os.path.join(root, filename)
            filelist.append(filepath)
            if filename.startswith("cat."):
                labels.append(0)
            else:
                labels.append(1)
    
        # shuffle
    tmp = np.array([filelist, labels])
    tmp = tmp.transpose()
    np.random.shuffle(tmp)
    
    filelist = list(tmp[:, 0])
    labels = list(tmp[:, 1])
    
    return filelist, labels


# convert dataset to tfrecord
def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def create_example(image, label):
    return tf.train.Example(features=tf.train.Features(feature={
                "image": bytes_feature(image),
                "label": int64_feature(label)
            }))


def tfrecord_pack(ds_dir, output_dir, output_name):
    output = os.path.join(output_dir, output_name + ".tfrecord")
    filelist, labels = get_filelist(ds_dir)
    nr_samples = len(labels)
    writer = tf.python_io.TFRecordWriter(output)
    
    for i in range(nr_samples):
        # resize image to (227, 227)
        image = Image.open(filelist[i])
        image = image.resize((224, 224))
        image = image.tobytes()
#         image = cv2.imread(filelist[i])
#         image = image.resize(image, (227, 227))
#         image = image.tostring()
        example = create_example(image, int(labels[i]))
        writer.write(example.SerializeToString())
        sys.stdout.write("\r>> Converting images to %s: %d/%d" % (output, i, nr_samples))
        sys.stdout.flush()
        
    print("")

In [3]:
tfrecord_pack("data/train", "data", "train")

>> Converting images to data\train.tfrecord: 24999/25000
